# Spark RDD

* Last updated 20161125 20170221
* spark.read.text vs spark.sparkContext.textFile

## S.1 학습내용

### S.1.1 목표

* Spark RDD를 사용할 수 있다.

### S.1.2 목차

* S.2 Data를 사용하는 API
* S.3 SparkSession
* S.3.1 Spark 2.0 vs 1.6
* S.3.2 SparkContext
* S.3.3 IPython Notebook에서 SparkSession 생성하기
* S.4 RDD
* S.5 Spark를 사용하기 전, Python 함수 사용하기
* S.6 Spark RDD 사용하기
* S.7 spark-submit

### S.1.3 문제

* 문제 S-1: Hello Spark - 환경설정을 읽어 클라이언트 sc를 생성하기.
* 문제 S-2: RDD를 사용하여 MLlib의 입력 데이터 word vector 생성하기.
* 문제 S-3: RDD를 사용하여 MLLib의 입력 데이터 feature vector 생성하기.

## S.2 Data를 사용하는 API

* 3가지 데이터
    * RDD, DataSet, DataFrame
* Spark의 RDD, DataFrame 모두 immutable
* Spark의 데이터는 모두 lazy (실제 transformation을 action까지 연기)
* Spark에서 모든 transformations이 연기된다는 lazy의 의미는:
    * 변환을 하여 메모리에 가지고 있는 비효율성
    * 실제 action이 실행되는 경우, 계산이 이루어짐.
    * RDD의 경우, action이 실행될 때마다 재계산이 이루어지는 것을 막기 위해 persist (or cache)함수를 사용할 수 있다.

데이터구조 | 도입된 spark version | 설명
---------|---------|---------
rdd | Spark 1.0 | unstructured, schema free, low-level
dataframe | 1.3 | semi 또는 structured, schema를 가진다. Dataset[Row]와 같은 의미로, 타잎을 강제하지 않는다.
dataset | 1.6 | 자바의 Generic과 같이 Dataset[T]으로 '타잎'을 강제하는 형식이다. Scala and Java에서 사용한다. Python loosely-typed이므로 사용하지 않는다.

## S.3 SparkSession

### S.3.1 Spark 2.0 vs 1.6

* Spark 2.0부터는 SparkSession으로 시작점을 통합해서, 개별적인 Context를 모두 통합해서 'pyspark.sql.SparkSession'를 사용한다.

```
spark = SparkSession.builder.getOrCreate()
```

Context 구분 | 설명 | 2.0의 사용 예 | 1.x의 사용 예
----------|----------|----------|----------
SparkContext | RDD를 사용하는 Context | spark.SparkContext | SparkContext()
StreamingContext | 향후 제공 | |
SQLContext | Spark SQL, DataFrame | spark.sql | SQLContext(sarkContext)
HiveContext | HiveQL, DataFrame | |

* 이전 1.x 버전에서는 SparkContext를 통해 다른 Context를 사용했고, 호환성을 제공하므로 그대로 사용할 수도 있다.
* Spark 1.6에서는 다음과 같이 사용한다.

```
import pyspark
conf=pyspark.SparkConf()
conf = pyspark.SparkConf().setAppName("myAppName")
sc = pyspark.SparkContext(conf=conf) #SparkContext를 직접 생성한다.
sqlContext = SQLContext(sc)   #SparkContext를 넣어서 SQLContext를 생성
```

### S.3.2 SparkContext

* Spark 서버(클러스터)에 대한 클라이언트와 같은 역할을 한다.
* 클러스터를 어떻게 사용할 것인지 정하는 것 -> cluster manager에서 system resource를 할당받음 (cpu, memory, machine)
* Python의 SparkContext는 jar를 분산환경에서 사용하게 되므로 주의 (Scala, Java와 다름)
    * pyFiles에 사용할 (의존적인) 라이브러리를 넣는다 (또는 사용할 라이브러리가 없으면 빈 파일로 둔다).

* Cannot run multiple SparkContexts at once;
    * sc가 이미 있는 경우
```
from pyspark import SparkContext
sc = SparkContext("master","my python app", sparkHome="sparkhome",pyFiles="placeholderdeps.zip")
```

* SparkConf
    * spark-defaults.conf와 같은 파일의 값을 읽어서 설정
```
scala> sc.getConf.getOption("spark.local.dir")
res0: Option[String] = None

scala> sc.getConf.getOption("spark.app.name")
res1: Option[String] = Some(Spark shell)

scala> sc.getConf.get("spark.master")
res2: String = local[*]
```

### S.3.3 IPython Notebook에서 SparkSession 생성하기

PySpark on IPython Notebook

* ipython notebook에서 pyspark를 사용
* kernel을 만들지 않고, findspark를 사용한다.


## 문제 S-1: Hello Spark: 환경설정을 읽어 클라이언트 sc를 생성하기.

* 설정을 변경한다.

설정 항목 | 설명
----------|----------
SPARK_HOME | Spark를 설치한 자신의 경로로 수정한다.
PYTHONPATH | sys.path.insert()를 사용하여 PYTHONPATH를 수정한다. pyspark.zip, py4j-0.10.1-src.zip를 추가


In [2]:
import os
import sys 
os.environ["SPARK_HOME"]=os.path.join(os.environ['HOME'],'Downloads','spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [3]:
for i in sys.path:
    print i

/home/jsl/Downloads/spark-2.0.0-bin-hadoop2.7/python/lib/pyspark.zip
/home/jsl/Downloads/spark-2.0.0-bin-hadoop2.7/python/lib/py4j-0.10.1-src.zip

/usr/lib/python2.7
/usr/lib/python2.7/plat-x86_64-linux-gnu
/usr/lib/python2.7/lib-tk
/usr/lib/python2.7/lib-old
/usr/lib/python2.7/lib-dynload
/home/jsl/.local/lib/python2.7/site-packages
/usr/local/lib/python2.7/dist-packages
/usr/lib/python2.7/dist-packages
/usr/lib/python2.7/dist-packages/PILcompat
/usr/lib/python2.7/dist-packages/gtk-2.0
/usr/lib/python2.7/dist-packages/ubuntu-sso-client
/usr/local/lib/python2.7/dist-packages/IPython/extensions
/home/jsl/.ipython


In [4]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

* 설정을 읽어 온다 (conf디렉토리 아래)  

In [3]:
print spark.version
print spark.conf.get('spark.app.name')
print spark.conf.get('spark.master')
print spark.conf.get('spark.driver.host')
#print spark.conf.get('spark.jars.packages')

2.0.0
pyspark-shell
local[*]
117.16.44.45


## S.4 RDD 소개

* RDD (Resilient Distributed Dataset)는 분산 레코드인 데이터 형식이다.
    * Resilient - fault tolerent (어느 한 노드에서 작업이 실패하면 다른 노드에서 실행된다.)
    * Distributed - multiple nodes in a clusters
    * Dataset - 데이터타잎으로 구성된다.
* RDD는 내, 외부 자료에서 생성하며, 생성된 자료는 read-only이다.
    * HDFS 파일을 처리할 수 있다.

* 3단계 처리
    * 1단계: 읽기 - 2가지 방식: 
        * 외부에서 읽기 
```
sparkContext.textFile()
```
        
        * 내부에서 읽기 parallelizing a collection
```
sparkContext.parallelize()
```
        
    * 2단계: 변환 transformations - lazy도 가능: RDD => RDD or seq(RDD)

함수 | 설명 | 예제
-------|-------|-------
map(fn) | 요소별로 fn을 적용해서 결과 rdd 돌려줌 | 
filter(fn) | 요소별로 선별하여 fn을 적용해서 결과 rdd 돌려줌 | filter(lambda line: "Spark" in line)
flatMap(fn) | 요소별로 fn을 적용하고, flat해서 결과 rdd 돌려줌 | .flatMap(lambda x: x.split(' '))
groupByKey() | key를 그룹해서 iterator를 돌려줌. |


    * 3단계: actions: RDD => a value (e.g., python list)

함수 | 설명 | 예제
-------|-------|-------
reduce(fn) | 요소별로 fn을 사용해서 줄여서 결과 list를 돌려줌 |
collect() | 모든 요소를 결과 list로 돌려줌 |
count() | 요소의 갯수를 결과 list로 돌려줌 |
countByKey() | |
foreach(fn) | |



## S.5 Spark를 사용하기 전, Python 함수 사용하기

* map, reduce, filter
    * 함수의 인자는 2개가 필요하다 (함수, 데이터).

함수 | 설명 | 예
-------|-------|-------
map() | 각 데이터 요소에 함수를 적용해서 list를 반환 | map(fn,data)
filter() | 각 데이터 요소에 함수의 결과 True를 선택해서 반환 | filter(fn, data)
reduce() | 각 데이터 요소에 함수를 적용해서 list를 반환 | reduce(fn, data)

* Python 함수로 처리한다.
    * 입출력은 데이터 하나씩이 아니라, list로 한다.

In [17]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c):
    f=list()
    for i in c:
        _f=(float(9)/5)*i + 32
        f.append(_f)
    return f

print c2f(celsius)

[102.56, 97.7, 99.14, 100.03999999999999]


* Python에서 제공하는 map() 함수를 사용한다. map() 함수의 인자:
    * (1) 함수명 (함수의 return은 반드시 있어야 한다.)
    * (2) 입력인자

In [18]:
celsius = [39.2, 36.5, 37.3, 37.8]

def c2f(c):
    return (float(9)/5)*c + 32

f=map(c2f, celsius)
print f

[102.56, 97.7, 99.14, 100.03999999999999]


* lambda함수를 사용한다.
    * lambda는 무명 함수이다. 처리 결과가 반환된다.

In [19]:
map(lambda c:(float(9)/5)*c + 32, celsius)

[102.56, 97.7, 99.14, 100.03999999999999]

* 문자열에 map()을 사용한다.

In [20]:
sentence = 'Hello World'
words = sentence.split()
print words

['Hello', 'World']


* 문자열을 사용하면, 각 단어를 split()한다.
* list를 사용하면, 각 요소를 split()한다.

In [21]:
sentence = "Hello World"
map(lambda x:x.split(),sentence)

[['H'], ['e'], ['l'], ['l'], ['o'], [], ['W'], ['o'], ['r'], ['l'], ['d']]

In [22]:
sentence = ["Hello World"]
map(lambda x:x.split(),sentence)

[['Hello', 'World']]

* filter()는 데이터를 선별한다.

In [37]:
fib = [0,1,1,2,3,5,8,13,21,34,55]
result = filter(lambda x: x % 2, fib)
print result

[1, 1, 3, 5, 13, 21, 55]


* reduce()는 2개의 인자를 받는다.
* [ func(func(s1, s2),s3), ... , sn ]와 같이 수행한다.

In [36]:
reduce(lambda x, y: x+y, range(1,101))

5050

## S.6 Spark RDD 사용하기

* Apache spark wiki에서 첫 문단을 복사해 왔다.
* 3째줄은 한글, 4째 줄은 같은 단어를 반복해 추가했다.


In [10]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Overwriting data/ds_spark_wiki.txt


* 파일에서 읽기
    ```
    textFile()
    ```

In [ ]:
textFile=spark.read.text("ds_spark_wiki.txt")

In [17]:
#textFile = sc.textFile("data/ds_spark_wiki.txt")
textFile = spark.sparkContext.textFile("data/ds_spark_wiki.txt")

In [18]:
textFile.first()

u'Wikipedia'

* map()함수로 단어 분리하기

In [19]:
words=textFile.map(lambda x:x.split(' '))

In [20]:
words.count()

9

* lambda아닌 함수로 map()

In [21]:
def mySplit(x):
    return x.split(" ")

words=textFile.map(mySplit)

In [22]:
words.count()

9

In [23]:
words.collect()

[[u'Wikipedia'],
 [u'Apache',
  u'Spark',
  u'is',
  u'an',
  u'open',
  u'source',
  u'cluster',
  u'computing',
  u'framework.'],
 [u'\uc544\ud30c\uce58',
  u'\uc2a4\ud30c\ud06c\ub294',
  u'\uc624\ud508',
  u'\uc18c\uc2a4',
  u'\ud074\ub7ec\uc2a4\ud130',
  u'\ucef4\ud4e8\ud305',
  u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.'],
 [u'Apache',
  u'Spark',
  u'Apache',
  u'Spark',
  u'Apache',
  u'Spark',
  u'Apache',
  u'Spark'],
 [u'Originally',
  u'developed',
  u'at',
  u'the',
  u'University',
  u'of',
  u'California,',
  u"Berkeley's",
  u'AMPLab,'],
 [u'the',
  u'Spark',
  u'codebase',
  u'was',
  u'later',
  u'donated',
  u'to',
  u'the',
  u'Apache',
  u'Software',
  u'Foundation,'],
 [u'which', u'has', u'maintained', u'it', u'since.'],
 [u'Spark',
  u'provides',
  u'an',
  u'interface',
  u'for',
  u'programming',
  u'entire',
  u'clusters',
  u'with'],
 [u'implicit', u'data', u'parallelism', u'and', u'fault-tolerance.']]

In [24]:
for i in words.collect():
    print i

[u'Wikipedia']
[u'Apache', u'Spark', u'is', u'an', u'open', u'source', u'cluster', u'computing', u'framework.']
[u'\uc544\ud30c\uce58', u'\uc2a4\ud30c\ud06c\ub294', u'\uc624\ud508', u'\uc18c\uc2a4', u'\ud074\ub7ec\uc2a4\ud130', u'\ucef4\ud4e8\ud305', u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.']
[u'Apache', u'Spark', u'Apache', u'Spark', u'Apache', u'Spark', u'Apache', u'Spark']
[u'Originally', u'developed', u'at', u'the', u'University', u'of', u'California,', u"Berkeley's", u'AMPLab,']
[u'the', u'Spark', u'codebase', u'was', u'later', u'donated', u'to', u'the', u'Apache', u'Software', u'Foundation,']
[u'which', u'has', u'maintained', u'it', u'since.']
[u'Spark', u'provides', u'an', u'interface', u'for', u'programming', u'entire', u'clusters', u'with']
[u'implicit', u'data', u'parallelism', u'and', u'fault-tolerance.']


* 각 문장의 철자 갯수를 센다.
    * 첫 문장 'Wiskipedia'는 9

In [25]:
textFile.map(lambda s:len(s)).collect()

[9, 59, 32, 51, 72, 71, 30, 64, 46]

* filter()

In [26]:
_sparkLine=textFile.filter(lambda line: "Spark" in line)

In [27]:
print _sparkLine.count()

4


* 한글은 앞에 u를 붙여준다.

In [28]:
_line = textFile.filter(lambda line: u"스파크" in line)

In [29]:
print _line.first()

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


* groupByKey()
    * groupByKey()는 key를 묶어준다. 따라서 iterator를 반환한다. mapValues(sum)을 하면 key별 합계를 구할 수 있다.

In [30]:
textFile\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(sum)\
    .collect()

[(u'and', 1),
 (u'\uc18c\uc2a4', 1),
 (u'is', 1),
 (u'Wikipedia', 1),
 (u'AMPLab,', 1),
 (u'maintained', 1),
 (u'donated', 1),
 (u'\ucef4\ud4e8\ud305', 1),
 (u'open', 1),
 (u'since.', 1),
 (u'for', 1),
 (u'\ud074\ub7ec\uc2a4\ud130', 1),
 (u'with', 1),
 (u'framework.', 1),
 (u'provides', 1),
 (u'Apache', 6),
 (u'Spark', 7),
 (u'was', 1),
 (u'Originally', 1),
 (u'which', 1),
 (u'fault-tolerance.', 1),
 (u'University', 1),
 (u'codebase', 1),
 (u'interface', 1),
 (u'data', 1),
 (u'\ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.', 1),
 (u'Foundation,', 1),
 (u'\uc624\ud508', 1),
 (u'programming', 1),
 (u'\uc2a4\ud30c\ud06c\ub294', 1),
 (u'the', 3),
 (u'entire', 1),
 (u'has', 1),
 (u'to', 1),
 (u'later', 1),
 (u'computing', 1),
 (u'Software', 1),
 (u'developed', 1),
 (u"Berkeley's", 1),
 (u'it', 1),
 (u'an', 2),
 (u'cluster', 1),
 (u'implicit', 1),
 (u'at', 1),
 (u'of', 1),
 (u'clusters', 1),
 (u'parallelism', 1),
 (u'\uc544\ud30c\uce58', 1),
 (u'California,', 1),
 (u'source', 1)]

* parallelize() 사용하기
    * list에서 읽어, rdd로 변환하기

In [32]:
_aList=[1,2,3]
rdd = spark.sparkContext.parallelize(_aList)

In [33]:
rdd.take(3)

[1, 2, 3]

* map(), collect() 사용해서 square

In [34]:
nums = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
print squared

[1, 4, 9, 16]


* 문장 처리하기
* 단어를 교체하기

In [35]:
a=["this is","a line"]
_rdd=spark.sparkContext.parallelize(a)

words=_rdd.map(lambda x:x.split())
print words.collect()

[['this', 'is'], ['a', 'line']]


In [36]:
_upper=_rdd.map(lambda x:x.replace("a","AA"))
_upper.take(10)

['this is', 'AA line']

* 첫 글자를 대문자로 만들어서 출력해 보기

In [37]:
's'.upper()

'S'

In [38]:
pluralRDD =words.map(lambda x: x[0].upper())
print pluralRDD.collect()

['THIS', 'A']


In [39]:
pluralRDD =words.map(lambda x: [i.upper() for i in x])
print pluralRDD.collect()

[['THIS', 'IS'], ['A', 'LINE']]


* transformation(map()), action(collect()) 함수를 한꺼번에

In [40]:
pluralRDD =words.map(lambda x: [i.upper() for i in x]).collect()
print pluralRDD

[['THIS', 'IS'], ['A', 'LINE']]


In [41]:
wordsLength = words\
    .map(len)\
    .collect()
print wordsLength

[2, 2]


* 파일에 쓰기

In [6]:
pluralRDD.saveAsTextFile("data/ds_spark_wiki1.txt")

* create RDD from CSV


In [6]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2


Writing ./data/ds_spark_2cols.csv


In [42]:
inp_file = spark.sparkContext.textFile("./data/ds_spark_2cols.csv")
numbers_rdd = inp_file.map(lambda line: line.split(','))

In [43]:
numbers_rdd.take(10)

[[u'35', u' 2'],
 [u'40', u' 27'],
 [u'12', u' 38'],
 [u'15', u' 31'],
 [u'21', u' 1'],
 [u'14', u' 19'],
 [u'46', u' 1'],
 [u'10', u' 34'],
 [u'28', u' 3'],
 [u'48', u' 1']]

In [ ]:
spark.conf.set()

## S.7 spark-submit

* sys.path 설정은 하지 않아도 된다.

In [50]:
%%writefile src/ds_spark_rdd_hello.py
import pyspark
#conf = pyspark.SparkConf().setAppName("myAppName1")
#sc   = pyspark.SparkContext(conf=conf)
#sc.setLogLevel("ERROR")
def doIt():
    print "---------RESULT-----------"
    print spark.version
    spark.conf.set("spark.logConf","false")
    rdd=spark.sparkContext.parallelize(range(1000), 10)
    print "mean=",rdd.mean()
    nums = spark.sparkContext.parallelize([1, 2, 3, 4])
    squared = nums.map(lambda x: x * x).collect()
    for num in squared:
        print "%i " % (num)

if __name__ == "__main__":
    myConf=pyspark.SparkConf()
    spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf=myConf)\
        .getOrCreate()
    doIt()
    spark.stop()
#spark.stop()

Overwriting src/ds_spark_rdd_hello.py


* spark-submit을 실행하기 전, 'conf/log4j.properties'를 수정 log level을 ERROR로 설정하였다.
```
log4j.rootCategory=ERROR, console
```

In [1]:
!/home/jsl/Downloads/spark-2.0.0-bin-hadoop2.7/bin/spark-submit src/ds_spark_rdd_hello.py

---------BEGIN-----------
---------RESULT-----------
2.0.0
 499.5
1 
4 
9 
16 
